In [50]:
import os
import gurobipy as gp
import numpy as np
import json

In [57]:
qubo_path = "./data/QUBO/qubo_Ns10_Nt9_Nq2_K15_gamma0.5_zeta0.1_rho0.1.json"

with open(qubo_path, "r") as f:
    qubo_data = json.load(f)

J = qubo_data["J"]
h = qubo_data["h"]

# Create a Gurobi model
model = gp.Model("QUBO_Model")
model.Params.TimeLimit = 60  # Set a time limit of 60 seconds

# Add binary variables
x = model.addVars(h.keys(), vtype=gp.GRB.BINARY, name="x")

# Set the objective function
obj = gp.quicksum(h[i] * x[i] for i in h.keys())
obj += gp.quicksum(J[k] * x[k.split(',')[0]] * x[k.split(',')[1]] for k in J.keys())

model.setObjective(obj, gp.GRB.MINIMIZE)

# Optimize the model
model.optimize()

Set parameter TimeLimit to value 60


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: Intel(R) Core(TM) i7-7820X CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  60

Academic license 2674347 - for non-commercial use only - registered to er___@kth.se
Optimize a model with 0 rows, 180 columns and 0 nonzeros
Model fingerprint: 0x5dff2017
Model has 2030 quadratic objective terms
Variable types: 0 continuous, 180 integer (180 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [9e-03, 3e-02]
  QObjective range [2e-03, 1e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000
Found heuristic solution: objective -0.0598917
Found heuristic solution: objective -0.9720653
Found heuristic solution: objective -0.9730572
Found heuristic solution: objective -0.9734430
Presolve tim

In [49]:
# Test cost of a bitstring
sample = [1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0]
def qubo_cost(bitstring, J, h):
    cost = 0
    for i in h.keys():
        cost += h[i] * bitstring[int(i)] 
    for k in J.keys():
        i, j = map(int, k.split(','))
        cost += J[k] * bitstring[i] * bitstring[j]
    return cost

print("Cost of sample:", qubo_cost(sample, J, h))

Cost of sample: -44.84347646959376


In [56]:
mip_path = "./data/MIP/MIP_Ns10_Nt9_Nq1_K15_gamma0.5_zeta0.1_rho0.1.json"
int_max = 2**1 - 1

with open(mip_path, "r") as f:
    mip_data = json.load(f)

J = mip_data["J"]
h = mip_data["h"]
h2 = mip_data["h2"]

# Create a Gurobi model
model = gp.Model("MIP_Model")
model.Params.TimeLimit = 60  # Set a time limit of 60 seconds

# Add integer variables
x = model.addVars(h.keys(), vtype=gp.GRB.INTEGER, name="x")

# x can only take values 0, 1, ..., 3
model.addConstrs((x[i] <= int_max for i in h.keys()), name="upper_bound")

# Set the objective function
obj = gp.quicksum(h[i] * x[i] for i in h.keys())
obj += gp.quicksum(h2[i] * x[i] * x[i] for i in h2.keys())
obj += gp.quicksum(J[k] * x[k.split(',')[0]] * x [k.split(',')[1]] for k in J.keys())

model.setObjective(obj, gp.GRB.MINIMIZE)

# Optimize the model
model.optimize()

Set parameter TimeLimit to value 60
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: Intel(R) Core(TM) i7-7820X CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  60

Academic license 2674347 - for non-commercial use only - registered to er___@kth.se
Optimize a model with 90 rows, 90 columns and 90 nonzeros
Model fingerprint: 0x1e010632
Model has 575 quadratic objective terms
Variable types: 0 continuous, 90 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 2e-02]
  QObjective range [2e-03, 3e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.0000000
Presolve removed 90 rows and 90 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work

In [9]:
# Load QUBO matrix from file
qubo_path = "./data/QUBO/qubo_Ns10_Nt9_Nq2_K15_gamma0.5_zeta0_rho0.txt"
Q = np.loadtxt(qubo_path)

# Create a Gurobi model
model = gp.Model("QUBO_Model")
model.Params.TimeLimit = 120  # Set a time limit of 60 seconds

# Define variables
x = model.addVars(Q.shape[0], vtype=gp.GRB.BINARY, name="x")

# Set objective
model.setObjective(gp.quicksum(Q[i, j] * x[i] * x[j] for i in range(Q.shape[0]) for j in range(Q.shape[1])), gp.GRB.MINIMIZE)

# Optimize model
model.optimize()

Set parameter TimeLimit to value 120
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: Intel(R) Core(TM) i7-7820X CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  120

Academic license 2674347 - for non-commercial use only - registered to er___@kth.se
Optimize a model with 0 rows, 180 columns and 0 nonzeros
Model fingerprint: 0x85aa7941
Model has 456 quadratic objective terms
Variable types: 0 continuous, 180 integer (180 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [4e-06, 1e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000
Found heuristic solution: objective -0.0601380
Found heuristic solution: objective -0.1516810
Presolve removed 0 rows and 180 columns
Presolve time: 0.00s
Presolve